In [1]:
import os
import numpy as np
import tensorflow as tf
# TensorFlow 로그 레벨 설정
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # 메모리 동적 할당
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # 메모리 제한
            )
    except RuntimeError as e:
        print(e)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


2024-12-29 05:57:59.737038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735451879.752939  525001 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735451879.757831  525001 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-29 05:57:59.774403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0000 00:00:1735451881.419475  525001 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libr

In [2]:
# 데이터 경로
data_dir = "/home/bcml1/sigenv/eeg_band_split" # npy 파일이 저장된 경로

# 데이터 및 라벨 로드 함수
def load_data(data_dir):
    data = []
    labels = []
    for file_name in sorted(os.listdir(data_dir)):
        if file_name.endswith("_bands.npy"):
            # 파일 경로
            file_path = os.path.join(data_dir, file_name)

            # 주파수 대역 데이터 로드
            try:
                band_data = np.load(file_path)  # (bands, samples, channels)
                alpha_band = band_data[1]  # 예: Alpha band 선택
            except Exception as e:
                print(f"Error loading {file_name}: {e}")
                continue

            # 파일 이름에서 라벨 추출
            label = "positive" if "positive" in file_name else "negative"

            # 데이터 및 라벨 추가
            for ch_idx in range(alpha_band.shape[1]):  # 채널별로 사용
                data.append(alpha_band[:, ch_idx])  # 채널 데이터 추가
                labels.append(label)

    return np.array(data), np.array(labels)


In [3]:
# Conv1D 기반 모델 정의
# cnn 모델을 구축하는 코드
def build_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        # Convolutional Layer 1
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # Convolutional Layer 2
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # Convolutional Layer 3
        Conv1D(128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # Flatten Layer
        Flatten(),

        # Fully Connected Dense Layers
        Dense(256, activation='relu'),
        Dropout(0.4),
        
        Dense(128, activation='relu'),
        Dropout(0.4),

        Dense(64, activation='relu'),
        Dropout(0.3),

        # Output Layer
        Dense(1, activation='sigmoid')  # 이진 분류
    ])
    return model

In [4]:
# 데이터 로드
X, y = load_data(data_dir)
print("Data shape:", X.shape)  # (samples, time)
print("Labels shape:", y.shape)

Data shape: (2576, 5120)
Labels shape: (2576,)


In [5]:
# 라벨 인코딩 (0: negative, 1: positive)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print("Encoded labels:", y[:5])


Encoded labels: [0 0 0 0 0]


In [6]:
# 데이터 정규화
X = X / np.max(np.abs(X), axis=1, keepdims=True)  # 정규화


In [7]:
# 데이터 차원 확장 (CNN 입력으로 사용)
X = np.expand_dims(X, axis=-1)  # (samples, time, 1)

In [8]:
# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)

Training data shape: (2060, 5120, 1)
Validation data shape: (516, 5120, 1)


In [9]:
# 모델 생성
input_shape = (X_train.shape[1], X_train.shape[2])  # (time, 1)
model = build_cnn_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/home/bcml1/sigenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32
)

Epoch 1/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 24s 348ms/step - accuracy: 0.5235 - loss: 0.7106 - val_accuracy: 0.4496 - val_loss: 0.6938
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 22s 341ms/step - accuracy: 0.5445 - loss: 0.6923 - val_accuracy: 0.4496 - val_loss: 0.6939
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 22s 340ms/step - accuracy: 0.5547 - loss: 0.6893 - val_accuracy: 0.5969 - val_loss: 0.6832
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 22s 333ms/step - accuracy: 0.6298 - loss: 0.6570 - val_accuracy: 0.6047 - val_loss: 0.6717
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 21s 330ms/step - accuracy: 0.7230 - loss: 0.5845 - val_accuracy: 0.6105 - val_loss: 0.6642
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 22s 332ms/step - accuracy: 0.7606 - loss: 0.5156 - val_accuracy: 0.6453 - val_loss: 0.6385
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 22s 334ms/step - accuracy: 0.8165 - loss: 0.4367 - val_accuracy: 0.6860 - val_loss: 0.5924
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 22s 340ms/step - accuracy: 0.8623 - loss: 0.3204 - 

In [11]:
# 학습 결과 저장
model.save("eeg_cnn_model_mj.h5")
print("Model saved!")

Model saved!


In [12]:
# 모델 로드 및 평가
model = tf.keras.models.load_model("eeg_cnn_model_mj.h5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7615 - loss: 1.0206 
Validation Loss: 1.0045523643493652, Validation Accuracy: 0.7790697813034058


In [13]:
# 테스트 데이터 예측
test_sample = X_val[0]  # 임의의 샘플
test_sample = np.expand_dims(test_sample, axis=0)  # 배치 차원 추가
prediction = model.predict(test_sample)

# 결과 출력
print(f"Prediction (0=negative, 1=positive): {prediction[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
Prediction (0=negative, 1=positive): 0.048786599189043045
